<a href="https://colab.research.google.com/github/kridtapon/TrendPulse-3X/blob/main/TrendPulse_3X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninst

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Supertrend Calculation
def calculate_supertrend(df, period=10, multiplier=3):
    """Calculate the Supertrend indicator."""
    hl2 = (df['High'] + df['Low']) / 2
    df['ATR'] = df['High'].rolling(window=period).apply(
        lambda x: max(x) - min(x), raw=True
    )
    df['Upperband'] = hl2 + (multiplier * df['ATR'])
    df['Lowerband'] = hl2 - (multiplier * df['ATR'])

    df['Supertrend'] = 0
    in_uptrend = True
    for current in range(1, len(df)):
        if df['Close'].iloc[current] > df['Upperband'].iloc[current - 1]:
            in_uptrend = True
        elif df['Close'].iloc[current] < df['Lowerband'].iloc[current - 1]:
            in_uptrend = False
        df['Supertrend'].iloc[current] = 1 if in_uptrend else -1

    return df

# ADX Calculation
def calculate_adx(df, period=14):
    """Calculate the Average Directional Index (ADX)."""
    high = df['High']
    low = df['Low']
    close = df['Close']

    df['TR'] = np.maximum.reduce([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ])

    df['+DM'] = np.where((high - high.shift(1)) > (low.shift(1) - low),
                         np.maximum(high - high.shift(1), 0), 0)
    df['-DM'] = np.where((low.shift(1) - low) > (high - high.shift(1)),
                         np.maximum(low.shift(1) - low, 0), 0)

    df['TR14'] = df['TR'].rolling(window=period).sum()
    df['+DM14'] = df['+DM'].rolling(window=period).sum()
    df['-DM14'] = df['-DM'].rolling(window=period).sum()

    df['+DI14'] = (df['+DM14'] / df['TR14']) * 100
    df['-DI14'] = (df['-DM14'] / df['TR14']) * 100
    df['DX'] = (abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14'])) * 100

    df['ADX'] = df['DX'].rolling(window=period).mean()
    return df

# Download data
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate indicators
df = calculate_supertrend(df, period=10, multiplier=1)
df = calculate_adx(df, period=10)

# Define entry and exit signals
adx_threshold = 40

df['Entry'] = (
    (df['Supertrend'] == 1) &  # Supertrend is green
    (df['ADX'] > adx_threshold) #&  # ADX > threshold
)

df['Exit'] = (
    (df['Supertrend'] == -1) &  # Supertrend is red
    (df['ADX'] > adx_threshold) #&  # ADX > threshold
)

# Filter data for test period (e.g., 2020-2025)
df_test = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Convert signals to boolean arrays
entries = df_test['Entry'].to_numpy()
exits = df_test['Exit'].to_numpy()

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df_test['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
<ipython-input-2-780430f29065>:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Supertrend'].iloc[current] = 1 if in_uptrend else -1
<ipython-input-2-780430f29065>:23: SettingWithCopyWa

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           643667.483629
Total Return [%]                       543.667484
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       2316.508472
Max Drawdown [%]                        25.242719
Max Drawdown Duration                       340.0
Total Trades                                    4
Total Closed Trades                             4
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 75.0
Best Trade [%]                         273.656557
Worst Trade [%]                         -6.331027
Avg Winning Trade [%]                  115.911767
Avg Losing Trade [%]                    -6.331027


In [6]:
# Download data
symbol = 'META'
start_date = '2020-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)
df_test = df['Close']
pf = vbt.Portfolio.from_holding(df_test, init_cash=100_000)
pf.total_profit()

[*********************100%***********************]  1 of 1 completed


180172.85559650476

In [7]:
pf.stats()

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,280172.855597
Total Return [%],180.172856
Benchmark Return [%],180.172856
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],76.73609


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Supertrend Calculation
def calculate_supertrend(df, period=10, multiplier=3):
    hl2 = (df['High'] + df['Low']) / 2
    df['ATR'] = df['High'].rolling(window=period).apply(
        lambda x: max(x) - min(x), raw=True
    )
    df['Upperband'] = hl2 + (multiplier * df['ATR'])
    df['Lowerband'] = hl2 - (multiplier * df['ATR'])

    df['Supertrend'] = 0
    in_uptrend = True
    for current in range(1, len(df)):
        if df['Close'].iloc[current] > df['Upperband'].iloc[current - 1]:
            in_uptrend = True
        elif df['Close'].iloc[current] < df['Lowerband'].iloc[current - 1]:
            in_uptrend = False
        df['Supertrend'].iloc[current] = 1 if in_uptrend else -1

    return df

# ADX Calculation
def calculate_adx(df, period=14):
    high = df['High']
    low = df['Low']
    close = df['Close']

    df['TR'] = np.maximum.reduce([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))])
    df['+DM'] = np.where((high - high.shift(1)) > (low.shift(1) - low), np.maximum(high - high.shift(1), 0), 0)
    df['-DM'] = np.where((low.shift(1) - low) > (high - high.shift(1)), np.maximum(low.shift(1) - low, 0), 0)

    df['TR14'] = df['TR'].rolling(window=period).sum()
    df['+DM14'] = df['+DM'].rolling(window=period).sum()
    df['-DM14'] = df['-DM'].rolling(window=period).sum()

    df['+DI14'] = (df['+DM14'] / df['TR14']) * 100
    df['-DI14'] = (df['-DM14'] / df['TR14']) * 100
    df['DX'] = (abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14'])) * 100

    df['ADX'] = df['DX'].rolling(window=period).mean()
    return df

# Download data
symbol = 'META'
start_date = '2014-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Grid search for parameter optimization
period_values = range(10, 41, 5)
multiplier_values = [1, 2, 3]
adx_threshold_values = range(10, 41, 5)

best_return = -np.inf
best_params = None

# Iterate through combinations of parameters
for period, multiplier, adx_threshold in product(period_values, multiplier_values, adx_threshold_values):
    # Calculate indicators
    df_temp = df.copy()
    df_temp = calculate_supertrend(df_temp, period=period, multiplier=multiplier)
    df_temp = calculate_adx(df_temp, period=14)

    # Define entry and exit signals
    df_temp['Entry'] = (
        (df_temp['Supertrend'] == 1) &  # Supertrend is green
        (df_temp['ADX'] > adx_threshold)  # ADX > threshold
    )

    df_temp['Exit'] = (
        (df_temp['Supertrend'] == -1) &  # Supertrend is red
        (df_temp['ADX'] > adx_threshold)  # ADX > threshold
    )

    # Filter data for test period (e.g., 2015-2025)
    df_test = df_temp[(df_temp.index.year >= 2015) & (df_temp.index.year <= 2025)]

    # Convert signals to boolean arrays
    entries = df_test['Entry'].to_numpy()
    exits = df_test['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_test['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Evaluate the performance (total return, Sharpe ratio, or other metric)
    total_return = portfolio.stats()['Total Return [%]']

    if total_return > best_return:
        best_return = total_return
        best_params = (period, multiplier, adx_threshold)

# Output the best parameters and corresponding return
print(f"Best Parameters: Period={best_params[0]}, Multiplier={best_params[1]}, ADX Threshold={best_params[2]}")
print(f"Best Total Return: {best_return}%")


Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-ac09f94eaefe>:23: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https

Best Parameters: Period=15, Multiplier=2, ADX Threshold=30
Best Total Return: 696.4406154023399%


In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Supertrend Calculation
def calculate_supertrend(df, period=10, multiplier=3):
    hl2 = (df['High'] + df['Low']) / 2
    df['ATR'] = df['High'].rolling(window=period).apply(
        lambda x: max(x) - min(x), raw=True
    )
    df['Upperband'] = hl2 + (multiplier * df['ATR'])
    df['Lowerband'] = hl2 - (multiplier * df['ATR'])

    df['Supertrend'] = 0
    in_uptrend = True
    for current in range(1, len(df)):
        if df['Close'].iloc[current] > df['Upperband'].iloc[current - 1]:
            in_uptrend = True
        elif df['Close'].iloc[current] < df['Lowerband'].iloc[current - 1]:
            in_uptrend = False
        df['Supertrend'].iloc[current] = 1 if in_uptrend else -1

    return df

# ADX Calculation
def calculate_adx(df, period=14):
    high = df['High']
    low = df['Low']
    close = df['Close']

    df['TR'] = np.maximum.reduce([high - low, abs(high - close.shift(1)), abs(low - close.shift(1))])
    df['+DM'] = np.where((high - high.shift(1)) > (low.shift(1) - low), np.maximum(high - high.shift(1), 0), 0)
    df['-DM'] = np.where((low.shift(1) - low) > (high - high.shift(1)), np.maximum(low.shift(1) - low, 0), 0)

    df['TR14'] = df['TR'].rolling(window=period).sum()
    df['+DM14'] = df['+DM'].rolling(window=period).sum()
    df['-DM14'] = df['-DM'].rolling(window=period).sum()

    df['+DI14'] = (df['+DM14'] / df['TR14']) * 100
    df['-DI14'] = (df['-DM14'] / df['TR14']) * 100
    df['DX'] = (abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14'])) * 100

    df['ADX'] = df['DX'].rolling(window=period).mean()
    return df

# Download data
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Grid search for parameter optimization
period_values = range(10, 41, 2)
multiplier_values = [1, 2, 3]
adx_period_values = range(10, 21, 2)  # Add range for ADX period optimization
adx_threshold_values = range(10, 41, 2)

best_return = -np.inf
best_params = None

# Iterate through combinations of parameters
for period, multiplier, adx_period, adx_threshold in product(period_values, multiplier_values, adx_period_values, adx_threshold_values):
    # Calculate indicators
    df_temp = df.copy()
    df_temp = calculate_supertrend(df_temp, period=period, multiplier=multiplier)
    df_temp = calculate_adx(df_temp, period=adx_period)  # Use optimized ADX period

    # Define entry and exit signals
    df_temp['Entry'] = (
        (df_temp['Supertrend'] == 1) &  # Supertrend is green
        (df_temp['ADX'] > adx_threshold)  # ADX > threshold
    )

    df_temp['Exit'] = (
        (df_temp['Supertrend'] == -1) &  # Supertrend is red
        (df_temp['ADX'] > adx_threshold)  # ADX > threshold
    )

    # Filter data for test period (e.g., 2020-2025)
    df_test = df_temp[(df_temp.index.year >= 2020) & (df_temp.index.year <= 2025)]

    # Convert signals to boolean arrays
    entries = df_test['Entry'].to_numpy()
    exits = df_test['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_test['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    # Evaluate the performance (total return, Sharpe ratio, or other metric)
    total_return = portfolio.stats()['Total Return [%]']

    if total_return > best_return:
        best_return = total_return
        best_params = (period, multiplier, adx_period, adx_threshold)

# Output the best parameters and corresponding return
print(f"Best Parameters: Period={best_params[0]}, Multiplier={best_params[1]}, ADX Period={best_params[2]}, ADX Threshold={best_params[3]}")
print(f"Best Total Return: {best_return}%")


Streaming output truncated to the last 5000 lines.
<ipython-input-1-b258c2b58836>:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Supertrend'].iloc[current] = 1 if in_uptrend else -1
<ipython-input-1-b258c2b58836>:23: SettingWithCopyWa

Best Parameters: Period=10, Multiplier=1, ADX Period=10, ADX Threshold=40
Best Total Return: 543.6674836294342%
